### 第六部分 元编程
#### 第十九章 动态属性和特性
##### 19.1 使用动态属性转换数据

In [2]:
# 下载osconfeed.json
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
    with open(JSON, encoding='utf-8') as fp:
        return json.load(fp)

In [16]:
feed = load()
print(sorted(feed['Schedule'].keys()))
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))
print(feed['Schedule']['speakers'][-1]['name'])
print(feed['Schedule']['speakers'][-1]['serial'])
print(feed['Schedule']['events'][40]['name'])
print(feed['Schedule']['events'][40]['speakers'])

['conferences', 'events', 'speakers', 'venues']
  1 conferences
494 events
357 speakers
 53 venues
Carina C. Zona
141590
There *Will* Be Bugs
[3471, 5199]


In [35]:
# 吧一个JSON数据集转换成一个嵌套着FrozenJSON对象、列表和简单类型的FrozenJSON对象
from collections import abc
import keyword

class FrozenJSON:
    '''一个只读接口， 使用属性表示法访问JSON类对象
    '''
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            try:
                return FrozenJSON.build(self.__data[name])
            except KeyError:
                raise AttributeError('没有这个属性！')
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [3]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
print(len(feed.Schedule.speakers))
print(sorted(feed.Schedule.keys()))
for key, value in sorted(feed.Schedule.items()):
    print('{:3} {}'.format(len(value), key))
print(feed.Schedule.speakers[-1].name)
talk = feed.Schedule.events[40]
print(type(talk))
print(talk.name, talk.speakers)
print(talk.flavor)

357
['conferences', 'events', 'speakers', 'venues']
  1 conferences
494 events
357 speakers
 53 venues
Carina C. Zona
<class '__main__.FrozenJSON'>
There *Will* Be Bugs [3471, 5199]


KeyError: 'flavor'

In [1]:
# 使用__new__方法取代build方法，构建可能是也可能不是FrozenJSON实例的新对象
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

In [6]:
# 访问保存在shelve.Shelf对象里的OSCON日程数据
import warnings

DB_NAME = 'data/schedule1.db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

In [7]:
# 导入模块，定义常量和增强的Record类
import warnings
import inspect

DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

In [8]:
# MissingDatabaseError类和DbRecord类
class MissingDatabaseError(RuntimeError):
    '''需要数据库但没有指定数据库时抛出'''

class DbRecord(Record):
    
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
    
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

In [9]:
# Event类
class Event(DbRecord):
    
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)
    
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speakers_objs = [fecth('speaker.{}'.format(key))
                                        for key in spkr_serials]
        return self._speakers_objs
    
    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()

In [12]:
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

##### 19.2 使用特性验证属性


In [3]:
# 最简单的LineItem类
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight= weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [12]:
raisins = LineItem('Golden raisins', 10, 6.95)
print(raisins.subtotal())
raisins.weight = -20
print(raisins.subtotal())

69.5


ValueError: value must be > 0

In [11]:
# 定义了weight特性的LineItem类
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self._weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self._weight = value
        else:
            raise ValueError('value must be > 0')

##### 19.3 特性全解析
虽然内置的 property 经常用作装饰器，但它其实是一个类。在 Python
中，函数和类通常可以互换，因为二者都是可调用的对象，而且没有实
例化对象的 new 运算符，所以调用构造方法与调用工厂函数没有区别。
此外，只要能返回新的可调用对象，代替被装饰的函数，二者都可以用
作装饰器。

In [29]:
# 实例属性覆盖类的数据属性
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [30]:
obj = Class()
print(vars(obj), obj.data)
obj.data = 'bar'
print(vars(obj))
print(obj.data, Class.data)

{} the class data attr
{'data': 'bar'}
bar the class data attr


In [31]:
# 实例属性不会覆盖类特性
print(Class.prop)
print(obj.prop)
obj.prop = 'foo'

the prop value


AttributeError: can't set attribute

In [32]:
obj.__dict__['prop'] = 'foo'
print(vars(obj))
print(obj.prop)
Class.prop = 'baz'
print(obj.prop)

{'data': 'bar', 'prop': 'foo'}
the prop value
foo


In [35]:
# 新添加的类特性覆盖现有的实例属性
print(obj.data)
print(Class.data)
Class.data = property(lambda self: 'the "data" prop value')
print(obj.data)

bar
the class data attr
the "data" prop value


In [36]:
del Class.data
print(obj.data)

bar


In [37]:
# 特性文档
class Foo:
    
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

In [39]:
print(help(Foo.bar))
print(help(Foo))

Help on property:

    The bar attribute

None
Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute

None


##### 19.4 定义一个特性工厂函数

In [40]:
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    
    return property(qty_getter, qty_setter)

In [41]:
class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weigth * self.price

In [46]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
print(nutmeg.weight, nutmeg.price)
print(sorted(vars(nutmeg).items()))

8 13.95
[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]


##### 19.5 处理属性删除操作

In [47]:
class BlackKnight:

    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]
    
    @property
    def member(self):
        print('next member is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [49]:
knight = BlackKnight()
print(knight.member)
del knight.member
del knight.member
del knight.member
del knight.member

next member is:
an arm
BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.
BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.
BLACK KNIGHT (loses a leg)
-- I'm invincible!
BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


##### 19.6 处理属性的重要属性和函数

In [56]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  '# 最简单的LineItem类\nclass LineItem:\n    \n    def __init__(self, description, weight, price):\n        self.description = description\n        self.weight= weight\n        self.price = price\n    \n    def subtotal(self):\n        return self.weight * self.price',
  "raisins = LineIteme('Golden raisins', 10, 6.95)\nprint(raisins.subtotal())",
  '# 最简单的LineItem类\nclass LineItem:\n    \n    def __init__(self, description, weight, price):\n        self.description = description\n        self.weight= weight\n        self.price = price\n    \n    def subtotal(self):\n        return self.weight * self.price',
  "raisins = LineIteme('Golden raisins', 10, 6.95)\nprint(raisins.subtotal())",
  "raisins = LineItem('Golden r

#### 第二十章 属性描述符
- 描述符是实现了特定协议的类，这个协议包括 \_\_get\_\_、\_\_set\_\_ 和\_\_delete\_\_ 方法。property 类实现了完整的描述符协议。通常，可以只实现部分协议。其实，我们在真实的代码中见到的大多数描述符只实现了 \_\_get\_\_ 和 \_\_set\_\_ 方法，还有很多只实现了其中的一个。

#####  20.1 描述符示例：验证属性

In [6]:
# 使用Quantity描述符管理LineItem的属性
class Quantity:
    
    def __init__(self, storage_name):
        self.storage_name = storage_name
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

In [6]:
# 自动获取存储属性的名称
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

In [7]:
class LineItem:
    #     weight = Quantity('weight')
    #     price = Quantity('price')
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [8]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

In [8]:
# 重构后的描述符类
import abc

class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

In [9]:
class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    
    @abc.abstractmethod
    def validate(self, instance, value):
        '''return validated value or raise ValueError'''

In [10]:
class Quantity(Validated):
    '''a number greater then zero'''
    
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value

In [11]:
class NonBlank(Validated):
    '''a string with at least one non-space character'''
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [13]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [19]:
truffle1 = LineItem('White truffle', 100, 1)
truffle2 = LineItem('White truffle', 100, 1)
print(dir(truffle1))
print(dir(truffle2))

['_NonBlank#0', '_Quantity#0', '_Quantity#1', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'description', 'price', 'subtotal', 'weight']
['_NonBlank#0', '_Quantity#0', '_Quantity#1', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'description', 'price', 'subtotal', 'weight']


##### 20.2 覆盖型与非覆盖型描述符对比

In [2]:
# 几个简单的类， 用于研究描述符的覆盖行为
# 辅助函数， 仅用于显示

def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_arg = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_arg))

In [3]:
class Overriding:
    '''也称数据描述符或强制描述符'''
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class OverridingNoGet:
    '''没有__get__方法的覆盖型描述符'''
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class NonOverriding:
    '''也称非数据描述符或覆盖型描述符'''
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

In [4]:
# 覆盖型描述符的行为
obj = Managed()
obj.over
Managed.over
obj.over = 7
obj.over
obj.__dict__['over'] = 8
print(vars(obj))
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
-> Overriding.__get__(<Overriding object>, None, <class Managed>)
-> Overriding.__set__(<Overriding object>, <Managed object>, 7)
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
{'over': 8}
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [5]:
# 没有__get__方法的覆盖型描述符
print(obj.over_no_get)
print(Managed.over_no_get)
obj.over_no_get = 7
print(obj.over_no_get)
obj.__dict__['over_no_get'] = 9
print(obj.over_no_get)
obj.over_no_get = 7
print(obj.over_no_get)

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
9
-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
9


In [6]:
# 非覆盖型描述符
obj = Managed()
print(obj.non_over)
obj.non_over = 7
print(obj.non_over)
Managed.non_over
del obj.non_over
print(obj.non_over)

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
None
7
-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)
-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
None


#####  20.3 方法是描述符
- 在类中定义的函数属于绑定方法（bound method），因为用户定义的函数都有 __get__ 方法，所以依附到类上时，就相当于描述符。

In [12]:
obj = Managed()
print(obj.spam)
print(Managed.spam)
obj.spam = 7
print(obj.spam)

<bound method Managed.spam of <__main__.Managed object at 0x000001F239595BE0>>
<function Managed.spam at 0x000001F2376F1620>
7


In [13]:
# Text类型， 继承自UserString类
import collections

class Text(collections.UserString):
    
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]

In [28]:
word = Text('forward')
print(repr(word))
print(repr(word.reverse()))
print(repr(Text.reverse(Text('backward'))))
print(type(Text.reverse), type(word.reverse))
print(list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')])))
print(Text.reverse.__get__(word))
print(Text.reverse.__get__(None, Text))
print(word.reverse)
print(repr(word.reverse.__self__))
print(word.reverse.__func__ is Text.reverse)

Text('forward')
Text('drawrof')
Text('drawkcab')
<class 'function'> <class 'method'>
['diaper', (30, 20, 10), Text('desserts')]
<bound method Text.reverse of Text('forward')>
<function Text.reverse at 0x000001F239AA8378>
<bound method Text.reverse of Text('forward')>
Text('forward')
True


#### 第二十一章 类元编程
- 类元编程是指在运行时创建或定制类的技艺。在 Python 中，类是一等对象，因此任何时候都可以使用函数新建类，而无需使用 class 关键字。类装饰器也是函数，不过能够审查、修改，甚至把被装饰的类替换成其他类。最后，元类是类元编程最高级的工具：使用元类可以创建具有某种特质的全新类种，例如我们见过的抽象基类。
- 除非开发框架，否则不要编写元类——然而，为了寻找乐趣，或者练习相关的概念，可以这么做。

##### 21.1 类工厂函数

In [3]:
# 一个简单的类工厂函数
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: # 不能调用.replace或.split方法
        pass  # 假定field_names本身就是标志符组成的序列
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
    
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)
    
    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i
                          in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(__slots__ = field_names,
                    __init__ = __init__,
                    __iter__ = __iter__,
                    __repr__ = __repr__)
    
    return type(cls_name, (object,), cls_attrs)

In [7]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
print(rex)
print(Dog.__mro__)

Dog(name='Rex', weight=30, owner='Bob')
(<class '__main__.Dog'>, <class 'object'>)


##### 定制描述符的类装饰器

In [12]:
# 一个类装饰器
def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
    return cls

In [13]:
@entity
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [15]:
print(LineItem.description.storage_name)

_NonBlank#description


##### 21.3 导入时和运行时比较

In [24]:
import evaltime

##### 21.4 元类基础知识
- 根据 Python 对象模型，类是对象，因此类肯定是另外某个类的实例。默认情况下，Python 中的类是 type 类的实例。也就是说，type 是大多数内置的类和用户定义的类的元类

In [17]:
print(type(LineItem), type.__class__)

<class 'type'> <class 'type'>


In [21]:
import collections
print(collections.abc.Iterable.__class__)
import abc
print(abc.ABCMeta.__class__)
print(abc.ABCMeta.__mro__)

<class 'abc.ABCMeta'>
<class 'type'>
(<class 'abc.ABCMeta'>, <class 'type'>, <class 'object'>)


#####  21.5 定制描述符的元类

In [30]:
# EntityMeta元类以及它的一个实例
import collections

class EntityMeta(type):
    '''元类，用于创建带有验证字段的业务实体'''
    
    @classmethod
    def __prepare__(cls, name, bases):
        return collections.OrderedDict()
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        cls._field_names = []
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                cls._field_names.append(key)

In [34]:
class Entity(metaclass=EntityMeta):
    '''带有验证字段的业务实体'''
    
    @classmethod
    def field_names(cls):
        for name in cls._field_names:
            yield name

In [32]:
class LineItem(Entity):
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weigth
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [33]:
print(LineItem.description.storage_name)

_NonBlank#description


##### 21.6 元类的特殊方法 \_\_prepare\_\_

In [35]:
for name in LineItem.field_names():
    print(name)

description
weight
price


In [36]:
print(LineItem.mro())

[<class '__main__.LineItem'>, <class '__main__.Entity'>, <class 'object'>]
